## 6.1 Refactoring Strategy

### 6.1.1Classic Starategy

    - 아래 예제에서 각각의 구체적인 전략은 discount()라는 method를 가진 class
    - Strategy object는 state(object attribute)가 없다 ( ex: class내부에서 쓰이는 attribute들이 하나도 없음. 예를 들어, product, quantity, price같은)

In [1]:
from collections import namedtuple
from abc import ABC, abstractmethod

In [2]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity

class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            disc = 0
        else:
            disc = self.promotion.discount(self)
        return self.total() - disc
    def __repr__(self):
        fmt = '<Order Total: {:.2f}, due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC): #Strategy: Abstract Base Class
    @abstractmethod
    def discount(self, order):
        return 0
        """Return disc_amt"""

class FidelityPromotion(Promotion):
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
class BulkItemPromotion(Promotion):
    def discount(self, order):
        disc_amt = 0
        for item in order.cart:
            if item.quantity >= 20:
                disc_amt += item.total() * 0.1
        return disc_amt
class LargeOrderPromotion(Promotion):
    def discount(self, order):
        distinct_items = [item.product for item in order.cart]
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0 
    
Customer = namedtuple('Customer', 'name fidelity')
john = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('apple', 5, 2.0), LineItem('berry', 30, 0.5), LineItem('Chocolate', 30, 0.2)]
longcart = [LineItem(str(item_code), 1, 1.0) for item_code in range(20)]
print(Order(ann, cart, FidelityPromotion()))
print(Order(john, cart, BulkItemPromotion()))
print(Order(john, cart, LargeOrderPromotion()))
print(Order(ann, longcart, FidelityPromotion()))
print(Order(john, longcart, BulkItemPromotion()))
print(Order(john, longcart, LargeOrderPromotion()))

<Order Total: 31.00, due: 29.45>
<Order Total: 31.00, due: 28.90>
<Order Total: 31.00, due: 31.00>
<Order Total: 20.00, due: 19.00>
<Order Total: 20.00, due: 20.00>
<Order Total: 20.00, due: 18.60>


### 6.1.2 Function-oriented Strategy

    -기존 방법에서 구체적인 strategy를 function으로 변경했고, Promotion이라는 abstract class를 제거함
    -Order() object를 생성할 때마다, Promotion() object를 생성할 필요 없음

In [3]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.quantity * self.price
    
class Order: # Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        self.__total = sum(lineitem.total() for lineitem in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            disc = 0
        else:
            disc = self.promotion(self)
        return self.total() - disc
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
def fidelity_promotion(order):
    return order.total() * 0.05 if order.customer.fidelity>=1000 else 0
def bulkitem_promotion(order):
    disc_amt = 0
    for lineitem in order.cart:
        if lineitem.quantity >= 20:
            disc_amt += lineitem.total() * 0.1
    return disc_amt
def largeorder_promotion(order):
    distinct_items = [lineitem.product for lineitem in order.cart]
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    else:
        return 0

Customer = namedtuple('Customer', 'name fidelity')
john = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('apple', 5, 2.0), LineItem('berry', 30, 0.5), LineItem('Chocolate', 30, 0.2)]
longcart = [LineItem(str(item_code), 1, 1.0) for item_code in range(20)]
# print(Order(ann, cart, fidelity_promotion))
# print(Order(john, cart, bulkitem_promotion))
# print(Order(john, cart, largeorder_promotion))
# print(Order(ann, longcart, fidelity_promotion))
# print(Order(john, longcart, bulkitem_promotion))
# print(Order(john, longcart, largeorder_promotion))

### 6.1.3 Find the best strategy : Simple approach

In [4]:
promos = [fidelity_promotion, bulkitem_promotion, largeorder_promotion]
def best_promotion(order):
    return max(promo(order) for promo in promos)
print(Order(ann, cart, best_promotion))
print(Order(john, cart, best_promotion))
print(Order(ann, longcart, best_promotion))
print(Order(john, longcart, best_promotion))

<Order total: 31.00 due: 28.90>
<Order total: 31.00 due: 28.90>
<Order total: 20.00 due: 18.60>
<Order total: 20.00 due: 18.60>


    -위 예제는 가독성이 좋고, 제대로 작동하지만, 일부 코드가 중복되어 있어 버그가 생길 수 있음
    - 새로운 할인 전략을 추가하려면, 함수를 만들고, 이를 promos에 추가하여야 함

### 6.1.4 Find the strategy in a module

    -globals(): 현재 전역 symbol table을 나타내는 dictionary 객체를 반환. 현재 모듈에 대한 내용을 담음

In [5]:
global_dict = globals()
# print(global_dict.keys())
print(global_dict['BulkItemPromotion'])
print(global_dict['ABC'])
print(global_dict['best_promotion'])
print(global_dict['bulkitem_promotion'])

<class '__main__.BulkItemPromotion'>
<class 'abc.ABC'>
<function best_promotion at 0x7f61c40b4b70>
<function bulkitem_promotion at 0x7f61c40b4c80>


In [8]:
promos = [global_dict[name] for name in global_dict if name.endswith('_promotion') and name != 'best_promotion']
def best_promotion(order):
    return max(promo(order) for promo in promos)
print(Order(ann, cart, best_promotion))
print(Order(john, cart, best_promotion))
print(Order(ann, longcart, best_promotion))
print(Order(john, longcart, best_promotion))

<Order total: 31.00 due: 28.90>
<Order total: 31.00 due: 28.90>
<Order total: 20.00 due: 18.60>
<Order total: 20.00 due: 18.60>
